# <font color='blue'>Weber Souza</font>
# <font color='blue'>Deep Learning Para Aplicações de IA com PyTorch e Lightning</font>

## <font color='blue'>Lab 4</font>
## <font color='blue'>Detecção de Câncer em Imagens com Deep Learning</font>

![DSA](imagens/Lab4.png)

## Instalando e Carregando os Pacotes

In [ ]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

In [ ]:
# Para atualizar um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install -U nome_pacote

# Para instalar a versão exata de um pacote, execute o comando abaixo no terminal ou prompt de comando:
# !pip install nome_pacote==versão_desejada

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook.

# Instala o pacote watermark. 
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
#!pip install -q -U watermark

In [ ]:
!pip install -q torch==2.2.0+cu118 --extra-index-url https://download.pytorch.org/whl/cu118

In [ ]:
!pip install -q torch==2.2.0

In [1]:
!pip install -q pytorch-lightning

In [ ]:
import torch
torch.zeros(1).cuda()

In [ ]:
import torch
x = torch.rand(5, 3)
print(x)

In [ ]:
torch.cuda.is_available()

In [2]:
# Imports

# Manipulação de dados e imagens
import os                             # Manipulacao Sistema
import cv2                            # Open CV  
import itertools                      # Interacao com todas imagens no disco
import matplotlib.pyplot as plt       # Diagrama/Grafico
import numpy as np                
import pandas as pd
from tqdm import tqdm                 # Permite criar uma barra de progressao
from glob import glob                 # Manipulacao imagens
from PIL import Image                 # ||
import warnings
warnings.filterwarnings('ignore')     # ignorar qualquer aviso

# Pytorch
import torch
from torch import nn, optim                            # Neural Network , Otimizador
from torch.autograd import Variable                    # Gradientes
from torch.utils.data import DataLoader, Dataset       # DataLoader
from torchvision import models, transforms             # Transformacao

# Scikit-learn - Machine Learning for Python
from sklearn.model_selection import train_test_split   # Dividir em treino e teste
from sklearn.metrics import confusion_matrix           # Avaliacao
from sklearn.metrics import classification_report      # Avaliacao

# Pacotes para o relatório de hardware
import gc                              
import types
import pkg_resources
import pytorch_lightning as pl

# Seed para reproduzir os mesmos resultados
np.random.seed(10)
torch.manual_seed(10)
torch.cuda.manual_seed(10)

In [3]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Weber Souza" --iversions

Author: Weber Souza

PIL              : 10.2.0
pytorch_lightning: 2.2.1
matplotlib       : 3.8.0
numpy            : 1.26.4
pandas           : 2.1.4
torch            : 2.2.1+cpu
cv2              : 4.9.0
torchvision      : 0.17.1



## Verificando o Hardware

In [4]:
# Relatório completo

# Verificando o dispositivo
processing_device = "cuda" if torch.cuda.is_available() else "cpu"

# Verificando se GPU pode ser usada (isso depende da plataforma CUDA estar instalada)
torch_aval = torch.cuda.is_available()

# Labels para o relatório de verificação
lable_1 = 'Visão Geral do Ambiente'
lable_2 = 'Se NVIDIA-SMI não for encontrado, então CUDA não está disponível'
lable_3 = 'Fim da Checagem'

# Função para verificar o que está importado nesta sessão
def get_imports():

    for name, val in globals().items():
        if isinstance(val, types.ModuleType):
            name = val.__name__.split(".")[0]

        elif isinstance(val, type):            
            name = val.__module__.split(".")[0]

        poorly_named_packages = {"PIL": "Pillow", "sklearn": "scikit-learn"}

        if name in poorly_named_packages.keys():
            name = poorly_named_packages[name]

        yield name

# Imports nesta sessão
imports = list(set(get_imports()))

# Loop para verificar os requerimentos
requirements = []
for m in pkg_resources.working_set:
    if m.project_name in imports and m.project_name!="pip":
        requirements.append((m.project_name, m.version))
        
# Pasta com os dados (quando necessário)
pasta_dados = r'dados'

print(f'{lable_1:-^100}')
print()
print(f"Device:", processing_device)
print(f"Pasta de Dados: ", pasta_dados)
print(f"Versões dos Pacotes Requeridos: ", requirements)
print(f"Dispositivo Que Será Usado Para Treinar o Modelo: ", processing_device)
print(f"CUDA Está Disponível? ", torch_aval)
print("Versão do PyTorch: ", torch.__version__)
print("Versão do Lightning: ", pl.__version__)
print()
print(f'{lable_2:-^100}\n')
!nvidia-smi
gc.collect()
print()
print(f"Limpando a Memória da GPU (se disponível): ", torch.cuda.empty_cache())
print("\nModelo da GPU:")
# Modelo da GPU usada
!nvidia-smi --query-gpu=name --format=csv,noheader
print(f'\n{lable_3:-^100}')

--------------------------------------Visão Geral do Ambiente---------------------------------------

Device: cpu
Pasta de Dados:  dados
Versões dos Pacotes Requeridos:  [('matplotlib', '3.8.0'), ('numpy', '1.26.4'), ('pandas', '2.1.4'), ('torchvision', '0.17.1'), ('tqdm', '4.65.0')]
Dispositivo Que Será Usado Para Treinar o Modelo:  cpu
CUDA Está Disponível?  False
Versão do PyTorch:  2.2.1+cpu
Versão do Lightning:  2.2.1

------------------Se NVIDIA-SMI não for encontrado, então CUDA não está disponível------------------

Thu Apr 18 17:42:37 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 522.06       Driver Version: 522.06       CUDA Version: 11.8     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |     

## Mapeamento das Imagens

Obtemos todos os caminhos das imagens e fazemos o match com as informações em HAM10000_metadata.csv.

In [ ]:
# Pasta com as imagens
pasta_imagens = 'input'

In [ ]:
# Obtém o caminho de cada imagem
caminho_imagens = glob(os.path.join(pasta_imagens, '*', '*.jpg'))

In [ ]:
# Cria um dicionário
dict_map_imagem_caminho = {os.path.splitext(os.path.basename(x))[0]: x for x in caminho_imagens}

In [ ]:
dict_map_imagem_caminho

In [ ]:
# Leitura do arquivo de metadados
df_original = pd.read_csv(os.path.join(pasta_imagens, 'HAM10000_metadata.csv'))

In [ ]:
df_original.head()

In [ ]:
# Adiciona o path
df_original['path'] = df_original['image_id'].map(dict_map_imagem_caminho.get)

In [ ]:
df_original.head()

https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/DBW86T

In [ ]:
# Tipos de lesões que serão analisadas
dict_tipo_lesao = {'nv': 'Melanocytic nevi',
                   'mel': 'dermatofibroma',
                   'bkl': 'Benign keratosis-like lesions ',
                   'bcc': 'Basal cell carcinoma',
                   'akiec': 'Actinic keratoses',
                   'vasc': 'Vascular lesions',
                   'df': 'Dermatofibroma'}

In [ ]:
df_original['cell_type'] = df_original['dx'].map(dict_tipo_lesao.get)

In [ ]:
df_original.head()

In [ ]:
df_original['cell_type_idx'] = pd.Categorical(df_original['cell_type']).codes

In [ ]:
df_original.head()

## Pré-Processamento

Realizaremos diversas tarefas de pré-processamento das imagens.

### Extraindo Média e Desvio Padrão das Imagens

Esta função é usada para calcular a média e o desvio padrão em todo o conjunto de dados e será usada para normalização das imagens de entrada.

In [ ]:
# Função para cálculo de média e desvio
def calcula_img_mean_std(image_paths):

    # Define altura e largura que usaremos nas imagens
    img_h, img_w = 224, 224
    
    # Listas de controle
    imgs = []
    means, stdevs = [], []

    # Loop de leitura e resize das imagens
    for i in tqdm(range(len(image_paths))):
        img = cv2.imread(image_paths[i])
        img = cv2.resize(img, (img_h, img_w))
        imgs.append(img)

    # Stack de imagens
    imgs = np.stack(imgs, axis=3)
    print(imgs.shape)

    # Normalização
    imgs = imgs.astype(np.float32) / 255.

    # Loop de cálculo da média e desvio
    for i in range(3):
        pixels = imgs[:, :, i, :].ravel()  
        means.append(np.mean(pixels))
        stdevs.append(np.std(pixels))

    # BGR --> RGB
    means.reverse()  
    stdevs.reverse()

    print("normMean = {}".format(means))
    print("normStd = {}".format(stdevs))
    
    return means, stdevs

In [ ]:
# Retorna a média e o padrão de cada canal RGB.
norm_mean, norm_std = calcula_img_mean_std(caminho_imagens)

### Preparação do Dataset de Validação

In [ ]:
# Vamos verificar quantas imagens estão associadas a cada lesion_id
df_undup = df_original.groupby('lesion_id').count()

In [ ]:
df_undup.head()

In [ ]:
# Agora filtramos lesion_ids que possuem apenas uma imagem associada 
df_undup = df_undup[df_undup['image_id'] == 1]

In [ ]:
# Reset do índice
df_undup.reset_index(inplace = True)

In [ ]:
df_undup.head()

In [ ]:
# Função para identificar lesion_ids que possuem imagens duplicadas e aqueles que possuem apenas uma imagem
def get_duplicates(x):
    unique_list = list(df_undup['lesion_id'])
    if x in unique_list:
        return 'unduplicated'
    else:
        return 'duplicated'

In [ ]:
# Cria uma nova coluna que seja uma cópia da coluna lesion_id
df_original['duplicates'] = df_original['lesion_id']

In [ ]:
df_original.sample()

In [ ]:
# Aplica a função a esta nova coluna
df_original['duplicates'] = df_original['duplicates'].apply(get_duplicates)

In [ ]:
df_original.sample(5)

In [ ]:
# Vamos contar as duplicatas
df_original['duplicates'].value_counts()

In [ ]:
# Agora filtramos as imagens que não têm duplicatas
df_undup = df_original[df_original['duplicates'] == 'unduplicated']

In [ ]:
df_undup.shape

In [ ]:
# Agora criamos um val set usando df_undup porque temos certeza de que nenhuma dessas imagens tem duplicatas
y = df_undup['cell_type_idx']
_, df_val = train_test_split(df_undup, test_size = 0.2, random_state = 101, stratify = y)

In [ ]:
df_val.shape

In [ ]:
df_val['cell_type_idx'].value_counts()

### Separação das Amostras de Treino e Validação

In [ ]:
# Esta função identifica se uma imagem faz parte do conjunto train ou val
def get_val_rows(x):
    val_list = list(df_val['image_id'])
    if str(x) in val_list:
        return 'val'
    else:
        return 'train'

In [ ]:
# Identifica treino ou validação
df_original['train_or_val'] = df_original['image_id']

In [ ]:
# Aplica a função a esta nova coluna
df_original['train_or_val'] = df_original['train_or_val'].apply(get_val_rows)

In [ ]:
# Filtra as linhas de treino
df_treino = df_original[df_original['train_or_val'] == 'train']

In [ ]:
print(len(df_treino))
print(len(df_val))

In [ ]:
df_treino['cell_type_idx'].value_counts()

In [ ]:
df_val['cell_type'].value_counts()

### Dataset Augmentation

Podemos ver que há um sério desequilíbrio de classe nos dados de treinamento. Para resolver esse problema usaremos Dataset Augmentation criando imagens sintéticas a partir das imagens originais. Leia o manual em pdf no Capítulo 7 do curso.

In [ ]:
# Taxa de dataset augmentation a ser usada em cada classe
data_aug_rate = [15,10,5,50,0,40,5]

In [ ]:
# Loop para o dataset augmentation
for i in range(7):
    
    if data_aug_rate[i]:
        
        # Equaliza a proporção de imagens por classe nos dados de treino
        # Geramos novas imagens multiplicando as imagens existentes pela taxa definida na lista de taxas
        df_treino = df_treino.append([df_treino.loc[df_treino['cell_type_idx'] == i,:]] * (data_aug_rate[i] - 1), 
                                     ignore_index = True)

In [ ]:
df_treino['cell_type'].value_counts()

In [ ]:
# Reset do índice
df_treino = df_treino.reset_index()

In [ ]:
df_treino.shape

### Preparação das Amostras de Treino, Validação e Teste

In [ ]:
# Podemos dividir o conjunto de validação em um conjunto de validação e um conjunto de teste
df_val, df_teste = train_test_split(df_val, test_size = 0.5)

In [ ]:
# Reset do índice
df_val = df_val.reset_index()
df_teste = df_teste.reset_index()

In [ ]:
df_val.shape

In [ ]:
df_teste.shape

## Modelagem

Nesta etapa vamos constuir o processo de modelagem com 3 arquiteturas de Deep Learning. Leia os manuais em pdf no Capítulo 7 do curso.

### Função de Inicialização do Modelo e Definição de Arquitetura com Transfer Learning

In [ ]:
# feature_extracting é um booleano que define se estamos fazendo um ajuste fino ou extração de recursos.
# Se feature_extracting = False, o modelo é ajustado e todos os parâmetros do modelo são atualizados.
# Se feature_extracting = True, apenas os parâmetros da última camada são atualizados, os outros permanecem fixos.
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [ ]:
# Função para inicializar diferentes arquiteturas de Deep Learning
def inicializa_modelo(model_name, num_classes, feature_extract, use_pretrained = True):

    model_ft = None
    input_size = 0

    # Usaremos o modelo resnet50
    if model_name == "resnet":
        
        # Tamanho (pixels) das imagens de entrada
        input_size = 224
        
        # Carregamos o modelo pré-treinado com todos os pesos
        model_ft = models.resnet50(pretrained = use_pretrained)
        
        # Treinamos o modelo e atualizamos os pesos durante o treinamento
        set_parameter_requires_grad(model_ft, feature_extract)
        
        # Define o número de atributos de entrada
        num_ftrs = model_ft.fc.in_features
        
        # Camada linear final para prever a probabilidade das 7 classes com as quais estamos trabalhando
        model_ft.fc = nn.Linear(num_ftrs, num_classes)

    # Usaremos o modelo Densenet121
    elif model_name == "densenet":
        
        # Tamanho (pixels) das imagens de entrada
        input_size = 224
        
        # Carregamos o modelo pré-treinado com todos os pesos
        model_ft = models.densenet121(pretrained = use_pretrained)
        
        # Treinamos o modelo e atualizamos os pesos durante o treinamento
        set_parameter_requires_grad(model_ft, feature_extract)
        
        # Define o número de atributos de entrada
        num_ftrs = model_ft.classifier.in_features
        
        # Camada linear final para prever a probabilidade das 7 classes com as quais estamos trabalhando
        model_ft.classifier = nn.Linear(num_ftrs, num_classes)

    # Usaremos o Inception V3
    elif model_name == "inception":
        
        # Tamanho (pixels) das imagens de entrada
        # Tenha cuidado, pois espera-se (299 x 299) para o tamanho das imagens e ainda tem saída auxiliar
        input_size = 299

        # Carregamos o modelo pré-treinado com todos os pesos
        model_ft = models.inception_v3(pretrained = use_pretrained)
        
        # Treinamos o modelo e atualizamos os pesos durante o treinamento
        set_parameter_requires_grad(model_ft, feature_extract)
        
        # Tratando a auxilary net da arquitetura Inceptio
        model_ft.aux_logits = False
        
        # Tratando a primary net
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)

    else:
        print("Modelo inválido...")
        exit()
        
    return model_ft, input_size

### Inicializando o Modelo Escolhido e Definindo Transformações

In [ ]:
# Modelo que será treinado
#nome_modelo = 'densenet'
#nome_modelo = 'resnet'
nome_modelo = 'inception'

In [ ]:
num_classes = 7

In [ ]:
# Vamos treinar o modelo e sempre atualizar os pesos
feature_extract = False

In [ ]:
# Inicializa o modelo
model_ft, input_size = inicializa_modelo(nome_modelo, num_classes, feature_extract, use_pretrained = True)

In [ ]:
# Define o device
device = processing_device

In [ ]:
# Coloca o modelo no device
model = model_ft.to(device)

In [ ]:
# Transformações das imagens de treino
transform_treino = transforms.Compose([transforms.Resize((input_size,input_size)),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.RandomVerticalFlip(),transforms.RandomRotation(20),
                                       transforms.ColorJitter(brightness = 0.1, contrast = 0.1, hue = 0.1),
                                       transforms.ToTensor(), transforms.Normalize(norm_mean, norm_std)])

In [ ]:
# Transformações das imagens de validação
transform_val = transforms.Compose([transforms.Resize((input_size,input_size)), 
                                    transforms.ToTensor(),
                                    transforms.Normalize(norm_mean, norm_std)])

### Criando os DataLoaders

In [ ]:
# Defina um organizador de dados para modelo PyTorch 
class OrganizaDados(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):

        X = Image.open(self.df['path'][index])
        y = torch.tensor(int(self.df['cell_type_idx'][index]))

        if self.transform:
            X = self.transform(X)

        return X, y

In [ ]:
# Organiza e transforma os dados de treino
set_treino = OrganizaDados(df_treino, transform = transform_treino)

In [ ]:
# Cria o dataloader de treino
loader_treino = DataLoader(set_treino, batch_size = 32, shuffle = True, num_workers = 4)

In [ ]:
# O mesmo em validação
set_val = OrganizaDados(df_val, transform = transform_val)
loader_val = DataLoader(set_val, batch_size = 32, shuffle = False, num_workers = 4)

In [ ]:
# O mesmo em teste
set_teste = OrganizaDados(df_teste, transform = transform_val)
loader_teste = DataLoader(set_teste, batch_size = 32, shuffle = False, num_workers = 4)

In [ ]:
# Usaremos o otimizador Adam
optimizer = optim.Adam(model.parameters(), lr = 1e-3)

In [ ]:
# Usaremos cross entropy loss como função de perda
criterion = nn.CrossEntropyLoss().to(device)

## Treinamento

Nesta etapa treinaremos o modelo.

### Funções Para o Loop de Treino e Validação

In [ ]:
# Função para calcular erro em treino e validação durante o treinamento
class CalculaMetricas(object):
    
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [ ]:
# Listas para erro e acurácia em treino
total_loss_train, total_acc_train = [],[]

In [ ]:
# Função de treino do modelo
def treina_modelo(treino_loader, model, criterion, optimizer, epoch):
    
    # Coloca o modelo em modo de treino
    model.train()
    
    # Inicializa objetos de cálculo de métricas
    train_loss = CalculaMetricas()
    train_acc = CalculaMetricas()
    
    # Iteração
    curr_iter = (epoch - 1) * len(treino_loader)
    
    # Loop de treino
    for i, data in enumerate(treino_loader):
        
        # Extra os dados
        images, labels = data
        
        # Tamanho da imagem
        N = images.size(0)
        
        # Coloca imagens e labels no device
        images = Variable(images).to(device)
        labels = Variable(labels).to(device)

        # Zera os gradientes
        optimizer.zero_grad()
        
        # Previsão do modelo
        outputs = model(images)

        # Erro do modelo
        loss = criterion(outputs, labels)
        
        # Backpropagation
        loss.backward()
        optimizer.step()
        
        # Obtem a previsão de maior probabilidade
        prediction = outputs.max(1, keepdim = True)[1]
        
        # Atualiza as métricas
        train_acc.update(prediction.eq(labels.view_as(prediction)).sum().item()/N)
        train_loss.update(loss.item())
        
        # Iteração
        curr_iter += 1
        
        # Print e update das métricas
        # A condição *** and curr_iter < 1000 *** pode ser removida se você quiser treinar com o dataset completo
        if (i + 1) % 100 == 0 and curr_iter < 1000:
            print('[epoch %d], [iter %d / %d], [train loss %.5f], [train acc %.5f]' % (epoch, 
                                                                                       i + 1, 
                                                                                       len(treino_loader), 
                                                                                       train_loss.avg, 
                                                                                       train_acc.avg))
            total_loss_train.append(train_loss.avg)
            total_acc_train.append(train_acc.avg)
            
    return train_loss.avg, train_acc.avg

In [ ]:
# Listas para erro e acurácia em validação
total_loss_val, total_acc_val = [],[]

In [ ]:
# Função para validação
def valida_modelo(val_loader, model, criterion, optimizer, epoch):
    
    # Coloca o modelo em modo de validação
    model.eval()
    
    # Inicializa objetos de cálculo de métricas
    val_loss = CalculaMetricas()
    val_acc = CalculaMetricas()
    
    # Validação
    with torch.no_grad():
        for i, data in enumerate(val_loader):
            
            images, labels = data
            
            N = images.size(0)
            
            images = Variable(images).to(device)
            
            labels = Variable(labels).to(device)

            outputs = model(images)
            
            prediction = outputs.max(1, keepdim = True)[1]

            val_acc.update(prediction.eq(labels.view_as(prediction)).sum().item()/N)

            val_loss.update(criterion(outputs, labels).item())

    print('------------------------------------------------------------')
    print('[epoch %d], [val loss %.5f], [val acc %.5f]' % (epoch, val_loss.avg, val_acc.avg))
    print('------------------------------------------------------------')
    
    return val_loss.avg, val_acc.avg

### Treinamento do Modelo

In [ ]:
# Hiperparâmetros
epoch_num = 3
best_val_acc = 0

In [ ]:
%%time
for epoch in range(1, epoch_num + 1):
    
    # Execute a função de treino
    loss_train, acc_train = treina_modelo(loader_treino, model, criterion, optimizer, epoch)
    
    # Executa a função de validação
    loss_val, acc_val = valida_modelo(loader_val, model, criterion, optimizer, epoch)
    
    # Calcula as métricas
    total_loss_val.append(loss_val)
    total_acc_val.append(acc_val)
    
    # Verifica a acurácia em validação
    if acc_val > best_val_acc:
        best_val_acc = acc_val
        print('*****************************************************')
        print('Melhor Resultado: [epoch %d], [val loss %.5f], [val acc %.5f]' % (epoch, loss_val, acc_val))
        print('*****************************************************')

## Avaliação do Modelo

In [ ]:
# Plot
fig = plt.figure(num = 2)
fig1 = fig.add_subplot(2,1,1)
fig2 = fig.add_subplot(2,1,2)
fig1.plot(total_loss_train, label = 'Erro em Treino')
fig1.plot(total_acc_train, label = 'Acurácia em Treino')
fig2.plot(total_loss_val, label = 'Erro em Validação')
fig2.plot(total_acc_val, label = 'Acurácia em Validação')
plt.legend()
plt.show()

In [ ]:
# Função de plot da confusion_matrix
def plot_confusion_matrix(cm, 
                          classes,
                          normalize = False,
                          title = 'Confusion matrix',
                          cmap = plt.cm.Blues):

    plt.imshow(cm, interpolation = 'nearest', cmap = cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment = "center",
                 color = "white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('Label Real')
    plt.xlabel('Label Previsto')

In [ ]:
# Avaliação do modelo com dados de teste
model.eval()
y_label = []
y_predict = []
with torch.no_grad():
    for i, data in enumerate(loader_teste):
        images, labels = data
        N = images.size(0)
        images = Variable(images).to(device)
        outputs = model(images)
        prediction = outputs.max(1, keepdim = True)[1]
        y_label.extend(labels.cpu().numpy())
        y_predict.extend(np.squeeze(prediction.cpu().numpy().T))

In [ ]:
# Cria a confusion matrix
confusion_mtx = confusion_matrix(y_label, y_predict)

In [ ]:
# Plot da confusion matrix
plot_labels = ['akiec', 'bcc', 'bkl', 'df', 'nv', 'vasc','mel']
plot_confusion_matrix(confusion_mtx, plot_labels)

In [ ]:
# Gera o relatório de classificação
report = classification_report(y_label, y_predict, target_names = plot_labels)
print(report)

In [ ]:
# Plot de erros por classe
label_frac_error = 1 - np.diag(confusion_mtx) / np.sum(confusion_mtx, axis = 1)
plt.bar(np.arange(7),label_frac_error)
plt.xlabel('Label Real')
plt.ylabel('Classificação Incorreta')

## Conclusão

- Conseguimos 0.87574 de acurácia com o modelo DenseNet.
- Conseguimos 0.86607 de acurácia com o modelo ResNet.
- Conseguimos 0.86533 de acurácia com o modelo Inception.

# Fim